In [101]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [102]:
data=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [103]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
data=np.array(data)
data

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [7, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]])

In [105]:
np.random.shuffle(data)

In [106]:
data

array([[2, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [8, 0, 0, ..., 0, 0, 0]])

In [107]:
data=data.T

In [108]:
n,m=data.shape
print(n,m)


785 42000


In [109]:
data_dev=data[:,0:1000]
print(data_dev.shape)
Y_dev=data_dev[0:1,:]
print(Y_dev.shape)
X_dev=data_dev[1:,:]
print(X_dev.shape)

(785, 1000)
(1, 1000)
(784, 1000)


In [110]:
data_train=data[:,1000:]
print(data_train.shape)
Y_train=data_train[0:1,:]
print(Y_train.shape)
X_train=data_train[1:,:]
print(X_train.shape)

(785, 41000)
(1, 41000)
(784, 41000)


In [111]:
a=[[1,2],[3,4],[5,6]]

In [112]:
#accessing the first column
a=np.array(a)
a[:,0:1]

array([[1],
       [3],
       [5]])

In [113]:
#accessing the first row 
a[0:1,:]

array([[1, 2]])

In [114]:
a.size

6

In [115]:
def init_params():
    W1 = np.random.rand(10, 784) * np.sqrt(2 / 784)
    b1 = np.zeros((10, 1))
    W2 = np.random.rand(10, 10) * np.sqrt(2 / 10)
    b2 = np.zeros((10, 1))
    W1=np.array(W1)
    W2=np.array(W2)
    b1=np.array(b1)
    b2=np.array(b2)
    return W1,b1,W2,b2

In [116]:
def reLu(Z):
    return(np.maximum(0, Z))

In [117]:

def softmax(z):
    z = z - np.max(z)
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z)


In [118]:
def forward_prop(W1,b1,W2,X,b2):
    Z1=W1.dot(X)+b1
    A1=reLu(Z1)
    Z2=W2.dot(A1)+b2
    A2=softmax(Z2)
    return A1,Z1,A2,Z2

In [119]:
def one_hot(Y):
    one_hot_Y=np.zeros((Y.size,Y.max()+1))
    one_hot_Y[np.arange(Y.size),Y]=1
    one_hot_Y=one_hot_Y.T
    return(one_hot_Y)

In [120]:
def deriv_relu(Z):
    return Z>0 #returns element wise True or False bool value

In [121]:
def back_prop(A1,Z1,W1,W2,A2,Z2,Y,X):
    m=Y.size
    one_hot_Y=one_hot(Y)
    dz2=A2-one_hot_Y
    dw2=1/m*(dz2.dot(A1.T))
    db2=1/m*(np.sum(dz2))


    dz1=W2.T.dot(dz2)*deriv_relu(Z1)  # while reversing the activations do element wise multiplication
    dw1=1/m*(dz1.dot(X.T)) # in the end we need a 10x748 matrix same dimension as w1
    db1=1/m*(dz1)

    return dw1,db1,dw2,db2
    
    

In [122]:
def update_params(W1,W2,b1,b2,dw1,dw2,db1,db2,alpha):
    W1=W1-alpha*(dw1)
    W2=W2-alpha*(dw2)
    b1=b1-alpha*(db1)
    b2=b2-alpha*(db2)
    return W1,W2,b1,b2

In [123]:
def get_predictions(A2):
    return(np.argmax(A2,0))

def get_accuracy(predictions,Y):
    return(np.sum(Y==predictions)/Y.size)

In [124]:
def gradient_descent(X,Y,iterations,alpha):
    W1,b1,W2,b2=init_params()
    for i in range(iterations):
        A1,Z1,A2,Z2=forward_prop(W1,b1,W2,X,b2)
        dw1,db1,dw2,db2=back_prop(A1,Z1,W1,W2,A2,Z2,Y,X)
        W1,W2,b1,b2=update_params(W1,W2,b1,b2,dw1,dw2,db1,db2,alpha)
        if (i%50)==0:
            print("iteration :", i)
            print("accuracy : ", get_accuracy(get_predictions(A2),Y))
    return W1,b1,W2,b2

In [125]:
W1,b1,W2,b2=gradient_descent(X_train,Y_train,500,0.1)

iteration : 0
accuracy :  0.09826829268292683
iteration : 50
accuracy :  0.09826829268292683


KeyboardInterrupt: 